<a href="https://colab.research.google.com/github/alzaem2002-ctrl/cli/blob/main/examples/jupyter/prompt-management/manage-prompts-with-sdk-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manage Prompts with SDK - Tutorial

You can manage prompts easily through Agenta's web UI. But sometimes you might want to do things programmatically instead of using the interface.

In this tutorial, we'll use the Agenta SDK to create a new prompt, commit changes, deploy them to production, then fetch their configuration and use it to call the LLM.

## Tutorial Overview

Before we begin, let's quickly review how Agenta versions prompts:

Agenta follows a structure similar to **git** for prompt versioning. Instead of having one commit history, it uses **multiple branches (called variants)** where changes can be committed, and **environments** where these changes can be deployed (and used in your application).

The workflow for deploying a change to production that we'll follow in this tutorial is:

1. Create a new variant
2. Commit a change to that variant
3. Deploy that commit (variant/version) to the production environment
4. Commit a new change to that variant
5. Fetch the config from that environment


## Install dependencies


In [1]:
pip install -U agenta openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.2/422.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.7/95.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 22.

## Setup

Before using the SDK, we need to initialize it using the `ag.init()` method, which takes the host (default `cloud.agenta.ai`) and the API key (not required for community editions):


In [2]:
import os
os.environ["AGENTA_HOST"] = "https://cloud.agenta.ai/"  # Default value, no need to set explicitly unless self-hosted
os.environ["AGENTA_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [3]:
import os
import agenta as ag
from getpass import getpass

# Initialize the SDK with your API key
api_key = os.getenv("AGENTA_API_KEY")
if not api_key:
    os.environ["AGENTA_API_KEY"] = getpass("Enter your Agenta API key: ")

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# Initialize the SDK
ag.init()


KeyboardInterrupt: 

## Creating a new prompt

We're going to create a new completion prompt called `topic-explainer`.


In [ ]:
# Creates an empty application
app = ag.AppManager.create(
    app_slug="topic-explainer",
    template_key="SERVICE:completion", # we define here the app type, choices are SERVICE:completion, SERVICE:chat, CUSTOM
)

print("Created application:")
print(app)


:::warning
The app created until now is empty. You cannot use it from the UI yet. You need to create a variant and commit changes to it to be able to use it (next section).
:::


## Creating a new variant

Variants are similar to branches in **git**. Any change to the prompt must first be committed to a variant. Here, we'll create a new variant and make our first commit to it using the `VariantManager.create` method:


In [ ]:
from agenta.sdk.types import PromptTemplate, Message, ModelConfig
from pydantic import BaseModel

# We need to create a Pydantic Model with a `prompt` field of type `PromptTemplate`
class Config(BaseModel):
    prompt: PromptTemplate

config = Config(
    prompt=PromptTemplate(
        messages=[
            Message(role="system", content="You are an assistant that provides concise answers"),
            Message(role="user", content="Explain {{topic}} in simple terms"),
        ],
        llm_config=ModelConfig(
            model="gpt-3.5-turbo",
            max_tokens=150,
            temperature=0.6,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        ),
        template_format="curly"
    )
)

# Create a new variant
variant = ag.VariantManager.create(
    parameters=config.model_dump(),
    app_slug="topic-explainer",
    variant_slug="new-variant"
)

print("Created variant:")
print(variant)


This command will create a new variant and initialize it with the first commit containing the provided parameters.

- **Parameters:**
  - `app_slug`: The unique slug of your application.
  - `variant_slug`: The unique slug of the new variant.
  - `parameters`: A dictionary containing the initial configuration parameters.


## Deploying changes to the production environment

To deploy our commit to an environment, use the `DeploymentManager.deploy` method.


In [ ]:
# Deploy the variant to the production environment
deployment = ag.DeploymentManager.deploy(
    app_slug="topic-explainer",
    variant_slug="new-variant",
    environment_slug="production",
)

print("Deployed variant to environment:")
print(deployment)


- **Parameters:**
  - `environment_slug`: The slug of the environment (`development`, `staging`, or `production`).

- **Notes:**
  - Deploying a variant without specifying a `variant_version` deploys the latest version.


## Committing a change to variant

We're now going to commit changes to our variant. Note that this will not modify the version in deployment!

To save changes to a variant (creating a new version), we are going to use the `VariantManager.commit` method with explicit parameters.


In [ ]:
config2 = Config(
    prompt=PromptTemplate(
        messages=[
            Message(role="system", content="You are an assistant that provides concise answers"),
            Message(role="user", content="Use Paul Graham style to explain {{topic}} in simple terms."),
        ],
        llm_config=ModelConfig(
            model="gpt-3.5-turbo",
            max_tokens=150,
            temperature=0.9,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        ),
        template_format="curly"
    )
)

# Commit the new version
variant = ag.VariantManager.commit(
    parameters=config2.model_dump(),
    app_slug="topic-explainer",
    variant_slug="new-variant"
)

print("Committed new version of variant:")
print(variant)


:::info Immutability
Each commit creates a new version of the variant. Versions are immutable once created.
:::


## Fetching the prompt in production

Now we'll fetch and use the prompt that's in production. Keep in mind that the production environment still references the first version of our variant. If we want it to reflect the latest changes, we'll need to deploy it again.


In [ ]:
# Fetch configuration from the production environment
config = ag.ConfigManager.get_from_registry(
    app_slug="topic-explainer",
    environment_slug="production"
)

print("Fetched configuration from production:")
print(config)


In [ ]:
# Using the configuration with OpenAI client
from openai import OpenAI

# Format the prompt with your topic
topic = "artificial intelligence"
prompt = PromptTemplate(**config["prompt"]).format(topic=topic)
client = OpenAI()

response = client.chat.completions.create(
    **prompt.to_openai_kwargs()
)

print(f"Response for topic '{topic}':")
print(response.choices[0].message.content)


## Next Steps

Now that you've learned how to manage configurations using the SDK, you can:

- Read the guide to explore more advanced features of the SDK
- Read how to reference prompts in your traces
- Read how to manage configuration for your workflows (chain of prompts, RAG..)

## Summary

In this tutorial, we've covered the complete workflow for managing prompts with the Agenta SDK:

1. ✅ Created a new application
2. ✅ Created a variant with initial configuration
3. ✅ Deployed the variant to production environment
4. ✅ Committed changes to create a new version
5. ✅ Fetched configuration from production and used it with OpenAI

This demonstrates the Git-like workflow that Agenta uses for prompt versioning, allowing you to manage different variants and deploy specific versions to different environments.
